<a href="https://colab.research.google.com/github/naman9810/Fake-News-Detection/blob/main/SCRAPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy
import pandas as pd
from google.colab import drive
import tensorflow as tf
import numpy as np

In [ ]:
def scrape(words, date_since, numtweet):
  tweets = tweepy.Cursor(api.search, q=words, lang="en",
						since=date_since, tweet_mode='extended').items(numtweet)
  list_tweets = [tweet for tweet in tweets]
  tweet_texts=[]
  for tweet in list_tweets:
    retweetcount = tweet.retweet_count
    hashtags = tweet.entities['hashtags']
    try:
      text = tweet.retweeted_status.full_text
    except AttributeError:
      text = tweet.full_text
    print(f"Tweet Text:{text}")
    tweet_texts.append(text)
  x={'tweets':tweet_texts}
  e_t=pd.DataFrame(x)
  return e_t,tweet_texts

In [ ]:
consumer_key = "mid9bG90uqcpmhsZqhn1BM7QX"
consumer_secret = "90SqzgW1J6QTmBM0Vnx5GSWUHhRRTGyovX26Mk6qxuxpt81wYU"
access_key = "1291629689556131840-wYf8Say4YUYhoEu3ujmNWltKQjfcHs"
access_secret = "aM3EgNQzCSnNL2dCIOoY8HSYQXlZUHvimYEndltkfwQv1"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [ ]:
print("Enter Twitter HashTag:")
words = input()
print('Enter Date since The Tweets are required(YYYY-MM-DD):')
date_since = input()
print('Enter the number of tweets to be scrapped:')
numtweet=int(input())
print('EXTRACTED TWEETS:-')
extracted_tweets,tweets=scrape(words, date_since, numtweet)

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(extracted_tweets['tweets'])
extracted_tweets['tweets']= tokenizer.texts_to_sequences(extracted_tweets['tweets'])

In [ ]:
len_text = [len(x) for x in extracted_tweets['tweets'].values]
len_text = np.array(len_text)
m=np.mean(len_text)
std=np.std(len_text)
MAX_TEXT=int(2*std+m)

In [ ]:
X= tf.keras.preprocessing.sequence.pad_sequences(extracted_tweets['tweets'],maxlen=MAX_TEXT,padding='post',
                                                            truncating='post')

In [ ]:
drive.mount("/content/gdrive")

In [ ]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/fake_news/saved_model/my_model.h5')

In [ ]:
predict=model.predict(X)
for i in range(len(predict)):
  if predict[i]<0.5:
    print(tweets[i],' : FAKE')
  else:
    print(tweets[i]," : REAL")
